# 취향 분석

In [1]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

from hyperopt import fmin, hp, tpe, Trials
from scipy.io import mmwrite

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
color_cloth_column = ['newClothId', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
        'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers']
color_user_column = ['userId', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers']


In [4]:
transaction = pd.read_json('./transaction_top.json')
cloth_meta = pd.read_json('./final_top.json')
user_meta = pd.read_json('./user_top.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [5]:
transaction = transaction[transaction.shopCnt > 1]
transaction.shape

(281090, 3)

In [6]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [7]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [8]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [9]:
user_meta = user_meta[color_user_column]

In [10]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,858064,0.666667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.333333
1,858065,0.750000,0.000000,0.250000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,858067,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.500000,0.0,0.000000,0.250000,0.0,0.000000,0.0,0.0,0.000000,0.250000
3,858070,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,858071,0.000000,0.000000,0.800000,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131310,1416935,0.071429,0.214286,0.142857,0.0,0.0,0.071429,0.000000,0.0,0.071429,0.285714,0.0,0.071429,0.0,0.0,0.071429,0.000000
131311,1416936,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.333333,0.000000,0.0,0.666667,0.0,0.0,0.000000,0.000000
131312,1416938,0.000000,0.000000,0.200000,0.0,0.0,0.000000,0.200000,0.0,0.000000,0.600000,0.0,0.000000,0.0,0.0,0.000000,0.000000
131313,1416946,0.166667,0.000000,0.416667,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.000000,0.083333


In [11]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('colorWhite')] = 0.3
new_user.loc[new_user_idx, ('colorGrey')] = 0.3
new_user.loc[new_user_idx, ('colorBlack')] = 0.4
new_user.loc[new_user_idx, ('colorRed')] = 0
new_user.loc[new_user_idx, ('colorPink')] = 0
new_user.loc[new_user_idx, ('colorOrange')] = 0
new_user.loc[new_user_idx, ('colorIvory')] = 0
new_user.loc[new_user_idx, ('colorYellow')] = 0
new_user.loc[new_user_idx, ('colorGreen')] = 0
new_user.loc[new_user_idx, ('colorBlue')] = 0
new_user.loc[new_user_idx, ('colorPurple')] = 0
new_user.loc[new_user_idx, ('colorBrown')] = 0
new_user.loc[new_user_idx, ('colorBeige')] = 0
new_user.loc[new_user_idx, ('colorJean')] = 0
new_user.loc[new_user_idx, ('colorPattern')] = 0
new_user.loc[new_user_idx, ('colorOthers')] = 0
new_user = new_user[['userId',  'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers']]
new_user

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,8299.0,0.3,0.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
colum = list(new_user.columns)
di = dict()
for i in range(len(colum)):
    di[colum[i]] = new_user[colum[i]][0]
col_list = []
for idx, key in enumerate(di):
    if idx > 0 and di[key] > 0.2:
        col_list.append(key)
col_list.insert(0, 'userId')
print(col_list)

['userId', 'colorWhite', 'colorGrey', 'colorBlack']


In [13]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [14]:
for i in range(1, len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)

In [15]:
sub_col = list(sub_user.columns)[1:]

In [16]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sum(sub_user[col_i] for col_i in sub_col)
sub_user = sub_user.sort_values('favorSum')

In [17]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[953548, 1355941, 1215915]


In [18]:
cloth_meta = cloth_meta[color_cloth_column]

In [19]:
cloth_meta

,newClothId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,1929966,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1927660131403,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1928209,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1927270,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249601,1413008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
249602,779965,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
249603,1353489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
249604,1607233,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [20]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [21]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

59819

In [22]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [23]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [24]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [25]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 131315
clothId의 유니크 개수 : 59819
user테이블의 userId의 유니크 개수 : 131315
cloth테이블의 clothId의 유니크 개수 : 59819


In [26]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['colorWhite'][i], user_meta['colorGrey'][i], user_meta['colorBlack'][i],
                        user_meta['colorRed'][i], user_meta['colorPink'][i], user_meta['colorOrange'][i],
                        user_meta['colorIvory'][i], user_meta['colorYellow'][i], user_meta['colorGreen'][i],
                        user_meta['colorBlue'][i], user_meta['colorPurple'][i], user_meta['colorBrown'][i], user_meta['colorBeige'][i],
                        user_meta['colorJean'][i], user_meta['colorPattern'][i], user_meta['colorOthers'][i]]) for i in range(user_meta.shape[0])]

In [27]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['colorWhite'][i], cloth_meta['colorGrey'][i], cloth_meta['colorBlack'][i],
                        cloth_meta['colorRed'][i], cloth_meta['colorPink'][i], cloth_meta['colorOrange'][i],
                        cloth_meta['colorIvory'][i], cloth_meta['colorYellow'][i], cloth_meta['colorGreen'][i],
                        cloth_meta['colorBlue'][i], cloth_meta['colorPurple'][i], cloth_meta['colorBrown'][i], cloth_meta['colorBeige'][i],
                        cloth_meta['colorJean'][i], cloth_meta['colorPattern'][i], cloth_meta['colorOthers'][i]]) for i in range(cloth_meta.shape[0])]

In [28]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [29]:
interactions, weights = dataset.build_interactions(transaction_source)

In [30]:
user_features = dataset.build_user_features(user_features_source)

In [31]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [32]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [33]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [34]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [35]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 20, lrn_rate: 0.05388, precision: 0.00300                                                                     
no_comp: 15, lrn_rate: 0.05904, precision: 0.00291                                                                     
no_comp: 15, lrn_rate: 0.09176, precision: 0.00293                                                                     
no_comp: 20, lrn_rate: 0.07606, precision: 0.00290                                                                     
no_comp: 10, lrn_rate: 0.05858, precision: 0.00297                                                                     
no_comp: 15, lrn_rate: 0.05904, precision: 0.00251                                                                     
no_comp: 10, lrn_rate: 0.07696, precision: 0.00290                                                                     
no_comp: 15, lrn_rate: 0.07306, precision: 0.00299                                                                     
no_comp: 10, lrn_rate: 0.06259, precisio

In [36]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.053 1


In [37]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.95it/s]


In [38]:
# 모델 저장하기
import pickle
with open('model_color_top.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [205]:
# 모델 불러오기
import pickle
with open('model_color_top.pickle', 'rb') as f:
    model_top = pickle.load(f)

In [206]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [207]:
rec_size_top = sample_recommendation(model_top, cloth_meta.idx, real_user)
rec_size_top

[2139133140300,
 1440587133960,
 1648578142971,
 1685307140900,
 1282892131862,
 2152337133146,
 1572660131416,
 1873849131956,
 1804101134853,
 1804104134824]

In [208]:
real_user

[953548, 1355941, 1215915]

In [209]:
cloth_meta.insert(18, 'what', 6, True)
user_meta.insert(17, 'what', 6, True)

In [210]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,1440513131686,50031,6
1,1930909131765,7019,6
2,1930909131764,7840,6
3,1930909131766,7374,6
4,1931411131771,9496,6
...,...,...,...
59814,1281638267701,31274,6
59815,1622670267773,9458,6
59816,1556851267795,47549,6
59817,1651849267836,3995,6


In [211]:
cloth_meta.to_json('color_top_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('color_top_user_meta.json', orient='records', force_ascii=False)

In [212]:
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,what
0,858064,0.666667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.333333,6
1,858065,0.750000,0.000000,0.250000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,6
2,858067,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.500000,0.0,0.000000,0.250000,0.0,0.000000,0.0,0.0,0.000000,0.250000,6
3,858070,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,6
4,858071,0.000000,0.000000,0.800000,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131310,1416935,0.071429,0.214286,0.142857,0.0,0.0,0.071429,0.000000,0.0,0.071429,0.285714,0.0,0.071429,0.0,0.0,0.071429,0.000000,6
131311,1416936,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.333333,0.000000,0.0,0.666667,0.0,0.0,0.000000,0.000000,6
131312,1416938,0.000000,0.000000,0.200000,0.0,0.0,0.000000,0.200000,0.0,0.000000,0.600000,0.0,0.000000,0.0,0.0,0.000000,0.000000,6
131313,1416946,0.166667,0.000000,0.416667,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.000000,0.083333,6


# Outer

In [39]:
transaction = pd.read_json('./transaction_outer.json')
cloth_meta = pd.read_json('./final_outer.json')
user_meta = pd.read_json('./user_outer.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [40]:
transaction = transaction[transaction.shopCnt > 1]
transaction.shape

(171253, 3)

In [41]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [42]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [43]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [44]:
user_meta = user_meta[color_user_column]

In [45]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,488503,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,488504,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,488506,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,488507,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,488509,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93245,858056,0.0,0.0,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.4,0.0,0.0,0.2
93246,858057,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0
93247,858058,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
93248,858060,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('colorWhite')] = 0.3
new_user.loc[new_user_idx, ('colorGrey')] = 0.3
new_user.loc[new_user_idx, ('colorBlack')] = 0.4
new_user.loc[new_user_idx, ('colorRed')] = 0
new_user.loc[new_user_idx, ('colorPink')] = 0
new_user.loc[new_user_idx, ('colorOrange')] = 0
new_user.loc[new_user_idx, ('colorIvory')] = 0
new_user.loc[new_user_idx, ('colorYellow')] = 0
new_user.loc[new_user_idx, ('colorGreen')] = 0
new_user.loc[new_user_idx, ('colorBlue')] = 0
new_user.loc[new_user_idx, ('colorPurple')] = 0
new_user.loc[new_user_idx, ('colorBrown')] = 0
new_user.loc[new_user_idx, ('colorBeige')] = 0
new_user.loc[new_user_idx, ('colorJean')] = 0
new_user.loc[new_user_idx, ('colorPattern')] = 0
new_user.loc[new_user_idx, ('colorOthers')] = 0
new_user = new_user[['userId',  'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers']]
new_user

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,8299.0,0.3,0.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
colum = list(new_user.columns)
di = dict()
for i in range(len(colum)):
    di[colum[i]] = new_user[colum[i]][0]
col_list = []
for idx, key in enumerate(di):
    if idx > 0 and di[key] > 0.2:
        col_list.append(key)
col_list.insert(0, 'userId')
print(col_list)

['userId', 'colorWhite', 'colorGrey', 'colorBlack']


In [48]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [49]:
for i in range(1, len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)

In [50]:
sub_col = list(sub_user.columns)[1:]

In [51]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sum(sub_user[col_i] for col_i in sub_col)
sub_user = sub_user.sort_values('favorSum')

In [52]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[764472, 811509, 539708]


In [53]:
cloth_meta = cloth_meta[color_cloth_column]

In [54]:
cloth_meta

,newClothId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,2401813,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,234005278848,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2379678,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,2360803,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2354237,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85636,2322135130506,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
85637,2322135130507,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
85638,2322135130508,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
85639,2352578,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [56]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

25916

In [57]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [58]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [59]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [60]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 93250
clothId의 유니크 개수 : 25916
user테이블의 userId의 유니크 개수 : 93250
cloth테이블의 clothId의 유니크 개수 : 25916


In [61]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['colorWhite'][i], user_meta['colorGrey'][i], user_meta['colorBlack'][i],
                        user_meta['colorRed'][i], user_meta['colorPink'][i], user_meta['colorOrange'][i],
                        user_meta['colorIvory'][i], user_meta['colorYellow'][i], user_meta['colorGreen'][i],
                        user_meta['colorBlue'][i], user_meta['colorPurple'][i], user_meta['colorBrown'][i], user_meta['colorBeige'][i],
                        user_meta['colorJean'][i], user_meta['colorPattern'][i], user_meta['colorOthers'][i]]) for i in range(user_meta.shape[0])]

In [62]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['colorWhite'][i], cloth_meta['colorGrey'][i], cloth_meta['colorBlack'][i],
                        cloth_meta['colorRed'][i], cloth_meta['colorPink'][i], cloth_meta['colorOrange'][i],
                        cloth_meta['colorIvory'][i], cloth_meta['colorYellow'][i], cloth_meta['colorGreen'][i],
                        cloth_meta['colorBlue'][i], cloth_meta['colorPurple'][i], cloth_meta['colorBrown'][i], cloth_meta['colorBeige'][i],
                        cloth_meta['colorJean'][i], cloth_meta['colorPattern'][i], cloth_meta['colorOthers'][i]]) for i in range(cloth_meta.shape[0])]

In [63]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [64]:
interactions, weights = dataset.build_interactions(transaction_source)

In [65]:
user_features = dataset.build_user_features(user_features_source)

In [66]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [67]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [68]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [69]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [70]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 15, lrn_rate: 0.05779, precision: 0.00585                                                                     
no_comp: 20, lrn_rate: 0.06278, precision: 0.00621                                                                     
no_comp: 10, lrn_rate: 0.06152, precision: 0.00509                                                                     
no_comp: 15, lrn_rate: 0.05309, precision: 0.00553                                                                     
no_comp: 15, lrn_rate: 0.09976, precision: 0.00527                                                                     
no_comp: 15, lrn_rate: 0.07109, precision: 0.00535                                                                     
no_comp: 10, lrn_rate: 0.08436, precision: 0.00586                                                                     
no_comp: 10, lrn_rate: 0.07312, precision: 0.00555                                                                     
no_comp: 20, lrn_rate: 0.07392, precisio

In [71]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.063 2


In [72]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.08it/s]


In [73]:
# 모델 저장하기
import pickle
with open('model_color_outer.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [426]:
# 모델 불러오기
import pickle
with open('model_color_outer.pickle', 'rb') as f:
    model_outer = pickle.load(f)

In [427]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [428]:
rec_size_top = sample_recommendation(model_outer, cloth_meta.idx, real_user)
rec_size_top

[238071082834,
 157135683282,
 117337981361,
 177840683604,
 1577133111047,
 208840683451,
 231905785481,
 208155079212,
 209117486230,
 157348798846]

In [429]:
real_user

[764472, 811509, 539708]

In [430]:
cloth_meta.insert(18, 'what', 7, True)
user_meta.insert(17, 'what', 7, True)

In [431]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,234005278848,25240,7
1,208155079211,20700,7
2,208155079212,15426,7
3,208155079213,23065,7
4,208155079214,1832,7
...,...,...,...
25911,2080918130502,10939,7
25912,2080918130504,7261,7
25913,2322135130506,21919,7
25914,2322135130507,20550,7


In [432]:
cloth_meta.to_json('color_outer_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('color_outer_user_meta.json', orient='records', force_ascii=False)

In [433]:
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,what
0,488503,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,7
1,488504,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,7
2,488506,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,7
3,488507,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,7
4,488509,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93245,858056,0.0,0.0,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.4,0.0,0.0,0.2,7
93246,858057,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,7
93247,858058,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,7
93248,858060,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,7


# Pants

In [74]:
transaction = pd.read_json('./transaction_pants.json')
cloth_meta = pd.read_json('./final_pants.json')
user_meta = pd.read_json('./user_pants.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [75]:
transaction = transaction[transaction.shopCnt > 1]
transaction.shape

(273862, 3)

In [76]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [77]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [78]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [79]:
user_meta = user_meta[color_user_column]

In [80]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,14135,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.666667,0.0,0.000000
1,14141,0.037037,0.148148,0.222222,0.0,0.0,0.0,0.000000,0.0,0.037037,0.000000,0.0,0.000000,0.185185,0.333333,0.0,0.037037
2,14146,0.000000,0.666667,0.333333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
3,14148,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.0,0.000000
4,14158,0.100000,0.300000,0.100000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.100000,0.200000,0.200000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115960,488479,0.000000,0.038462,0.192308,0.0,0.0,0.0,0.115385,0.0,0.115385,0.000000,0.0,0.076923,0.153846,0.192308,0.0,0.115385
115961,488483,0.000000,0.200000,0.800000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
115962,488490,0.000000,0.454545,0.090909,0.0,0.0,0.0,0.181818,0.0,0.000000,0.090909,0.0,0.000000,0.090909,0.090909,0.0,0.000000
115963,488491,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000


In [81]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('colorWhite')] = 0.3
new_user.loc[new_user_idx, ('colorGrey')] = 0.3
new_user.loc[new_user_idx, ('colorBlack')] = 0.4
new_user.loc[new_user_idx, ('colorRed')] = 0
new_user.loc[new_user_idx, ('colorPink')] = 0
new_user.loc[new_user_idx, ('colorOrange')] = 0
new_user.loc[new_user_idx, ('colorIvory')] = 0
new_user.loc[new_user_idx, ('colorYellow')] = 0
new_user.loc[new_user_idx, ('colorGreen')] = 0
new_user.loc[new_user_idx, ('colorBlue')] = 0
new_user.loc[new_user_idx, ('colorPurple')] = 0
new_user.loc[new_user_idx, ('colorBrown')] = 0
new_user.loc[new_user_idx, ('colorBeige')] = 0
new_user.loc[new_user_idx, ('colorJean')] = 0
new_user.loc[new_user_idx, ('colorPattern')] = 0
new_user.loc[new_user_idx, ('colorOthers')] = 0
new_user = new_user[['userId',  'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers']]
new_user

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,8299.0,0.3,0.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
colum = list(new_user.columns)
di = dict()
for i in range(len(colum)):
    di[colum[i]] = new_user[colum[i]][0]
col_list = []
for idx, key in enumerate(di):
    if idx > 0 and di[key] > 0.2:
        col_list.append(key)
col_list.insert(0, 'userId')
print(col_list)

['userId', 'colorWhite', 'colorGrey', 'colorBlack']


In [83]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [84]:
for i in range(1, len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)

In [85]:
sub_col = list(sub_user.columns)[1:]

In [86]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sum(sub_user[col_i] for col_i in sub_col)
sub_user = sub_user.sort_values('favorSum')

In [87]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[118507, 466089, 324139]


In [88]:
cloth_meta = cloth_meta[color_cloth_column]

In [89]:
cloth_meta

,newClothId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,1971752,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,726566,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2213618,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2265443,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2005302,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106873,1889420,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106874,1275944,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106875,1889504,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106876,552678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [90]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [91]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

33530

In [92]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [93]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [94]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [95]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 115965
clothId의 유니크 개수 : 33530
user테이블의 userId의 유니크 개수 : 115965
cloth테이블의 clothId의 유니크 개수 : 33530


In [96]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['colorWhite'][i], user_meta['colorGrey'][i], user_meta['colorBlack'][i],
                        user_meta['colorRed'][i], user_meta['colorPink'][i], user_meta['colorOrange'][i],
                        user_meta['colorIvory'][i], user_meta['colorYellow'][i], user_meta['colorGreen'][i],
                        user_meta['colorBlue'][i], user_meta['colorPurple'][i], user_meta['colorBrown'][i], user_meta['colorBeige'][i],
                        user_meta['colorJean'][i], user_meta['colorPattern'][i], user_meta['colorOthers'][i]]) for i in range(user_meta.shape[0])]

In [97]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['colorWhite'][i], cloth_meta['colorGrey'][i], cloth_meta['colorBlack'][i],
                        cloth_meta['colorRed'][i], cloth_meta['colorPink'][i], cloth_meta['colorOrange'][i],
                        cloth_meta['colorIvory'][i], cloth_meta['colorYellow'][i], cloth_meta['colorGreen'][i],
                        cloth_meta['colorBlue'][i], cloth_meta['colorPurple'][i], cloth_meta['colorBrown'][i], cloth_meta['colorBeige'][i],
                        cloth_meta['colorJean'][i], cloth_meta['colorPattern'][i], cloth_meta['colorOthers'][i]]) for i in range(cloth_meta.shape[0])]

In [98]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [99]:
interactions, weights = dataset.build_interactions(transaction_source)

In [100]:
user_features = dataset.build_user_features(user_features_source)

In [101]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [102]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [103]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [104]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [105]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 10, lrn_rate: 0.09802, precision: 0.00474                                                                     
no_comp: 20, lrn_rate: 0.09377, precision: 0.00482                                                                     
no_comp: 10, lrn_rate: 0.07712, precision: 0.00497                                                                     
no_comp: 15, lrn_rate: 0.05801, precision: 0.00490                                                                     
no_comp: 10, lrn_rate: 0.05038, precision: 0.00494                                                                     
no_comp: 15, lrn_rate: 0.07198, precision: 0.00490                                                                     
no_comp: 20, lrn_rate: 0.09905, precision: 0.00449                                                                     
no_comp: 20, lrn_rate: 0.08900, precision: 0.00504                                                                     
no_comp: 20, lrn_rate: 0.05613, precisio

In [106]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.056 2


In [107]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.65it/s]


In [108]:
# 모델 저장하기
import pickle
with open('model_color_pants.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [467]:
# 모델 불러오기
import pickle
with open('model_color_pants.pickle', 'rb') as f:
    model_pants = pickle.load(f)

In [468]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [469]:
rec_size_top = sample_recommendation(model_pants, cloth_meta.idx, real_user)
rec_size_top

[19409615513,
 21801229810,
 141285112055,
 214926918340,
 23422909921,
 17611826560,
 15632909556,
 196639011167,
 232210520748,
 166869912945]

In [470]:
real_user

[118507, 466089, 324139]

In [471]:
cloth_meta.insert(18, 'what', 8, True)
user_meta.insert(17, 'what', 8, True)

In [472]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,16225354884,11720,8
1,14988915210,15601,8
2,19409615513,29005,8
3,20193975542,22897,8
4,18466875743,17729,8
...,...,...,...
33525,180323169094,6877,8
33526,182037869103,19148,8
33527,194119769137,18084,8
33528,179115969145,16294,8


In [473]:
cloth_meta.to_json('color_pants_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('color_pants_user_meta.json', orient='records', force_ascii=False)

In [474]:
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,what
0,14135,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.666667,0.0,0.000000,8
1,14141,0.037037,0.148148,0.222222,0.0,0.0,0.0,0.000000,0.0,0.037037,0.000000,0.0,0.000000,0.185185,0.333333,0.0,0.037037,8
2,14146,0.000000,0.666667,0.333333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,8
3,14148,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.0,0.000000,8
4,14158,0.100000,0.300000,0.100000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.100000,0.200000,0.200000,0.0,0.000000,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115960,488479,0.000000,0.038462,0.192308,0.0,0.0,0.0,0.115385,0.0,0.115385,0.000000,0.0,0.076923,0.153846,0.192308,0.0,0.115385,8
115961,488483,0.000000,0.200000,0.800000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,8
115962,488490,0.000000,0.454545,0.090909,0.0,0.0,0.0,0.181818,0.0,0.000000,0.090909,0.0,0.000000,0.090909,0.090909,0.0,0.000000,8
115963,488491,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,8


# Onepiece

In [475]:
transaction = pd.read_json('./transaction_onepiece.json')
cloth_meta = pd.read_json('./final_onepiece.json')
user_meta = pd.read_json('./user_onepiece.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [476]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [477]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [478]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [479]:
user_meta = user_meta[color_user_column]

In [480]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,8297,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8298,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8299,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5832,14129,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5833,14130,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5834,14131,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5835,14132,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [481]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('colorWhite')] = 0.3
new_user.loc[new_user_idx, ('colorGrey')] = 0.3
new_user.loc[new_user_idx, ('colorBlack')] = 0.4
new_user.loc[new_user_idx, ('colorRed')] = 0
new_user.loc[new_user_idx, ('colorPink')] = 0
new_user.loc[new_user_idx, ('colorOrange')] = 0
new_user.loc[new_user_idx, ('colorIvory')] = 0
new_user.loc[new_user_idx, ('colorYellow')] = 0
new_user.loc[new_user_idx, ('colorGreen')] = 0
new_user.loc[new_user_idx, ('colorBlue')] = 0
new_user.loc[new_user_idx, ('colorPurple')] = 0
new_user.loc[new_user_idx, ('colorBrown')] = 0
new_user.loc[new_user_idx, ('colorBeige')] = 0
new_user.loc[new_user_idx, ('colorJean')] = 0
new_user.loc[new_user_idx, ('colorPattern')] = 0
new_user.loc[new_user_idx, ('colorOthers')] = 0
new_user = new_user[['userId',  'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers']]
new_user

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,8299.0,0.3,0.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [482]:
colum = list(new_user.columns)
di = dict()
for i in range(len(colum)):
    di[colum[i]] = new_user[colum[i]][0]
col_list = []
for idx, key in enumerate(di):
    if idx > 0 and di[key] > 0.2:
        col_list.append(key)
col_list.insert(0, 'userId')
print(col_list)

['userId', 'colorWhite', 'colorGrey', 'colorBlack']


In [484]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [485]:
for i in range(1, len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)

In [486]:
sub_col = list(sub_user.columns)[1:]

In [487]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sum(sub_user[col_i] for col_i in sub_col)
sub_user = sub_user.sort_values('favorSum')

In [488]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[10780, 12086, 13922]


In [489]:
cloth_meta = cloth_meta[color_cloth_column]

In [490]:
cloth_meta

,newClothId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,1475941,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1995082,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1723870,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,2197982,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,2247825,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9870,2324195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9871,14649944700,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9872,2352427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9873,1457626,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [491]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [492]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

2047

In [493]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [494]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [495]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [496]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 5837
clothId의 유니크 개수 : 2047
user테이블의 userId의 유니크 개수 : 5837
cloth테이블의 clothId의 유니크 개수 : 2047


In [497]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['colorWhite'][i], user_meta['colorGrey'][i], user_meta['colorBlack'][i],
                        user_meta['colorRed'][i], user_meta['colorPink'][i], user_meta['colorOrange'][i],
                        user_meta['colorIvory'][i], user_meta['colorYellow'][i], user_meta['colorGreen'][i],
                        user_meta['colorBlue'][i], user_meta['colorPurple'][i], user_meta['colorBrown'][i], user_meta['colorBeige'][i],
                        user_meta['colorJean'][i], user_meta['colorPattern'][i], user_meta['colorOthers'][i]]) for i in range(user_meta.shape[0])]

In [498]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['colorWhite'][i], cloth_meta['colorGrey'][i], cloth_meta['colorBlack'][i],
                        cloth_meta['colorRed'][i], cloth_meta['colorPink'][i], cloth_meta['colorOrange'][i],
                        cloth_meta['colorIvory'][i], cloth_meta['colorYellow'][i], cloth_meta['colorGreen'][i],
                        cloth_meta['colorBlue'][i], cloth_meta['colorPurple'][i], cloth_meta['colorBrown'][i], cloth_meta['colorBeige'][i],
                        cloth_meta['colorJean'][i], cloth_meta['colorPattern'][i], cloth_meta['colorOthers'][i]]) for i in range(cloth_meta.shape[0])]

In [499]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [500]:
interactions, weights = dataset.build_interactions(transaction_source)

In [501]:
user_features = dataset.build_user_features(user_features_source)

In [502]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [503]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [504]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [505]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [506]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 10, lrn_rate: 0.08950, precision: 0.00972    
no_comp: 20, lrn_rate: 0.06037, precision: 0.01076                                 
no_comp: 15, lrn_rate: 0.05629, precision: 0.01061                                 
no_comp: 20, lrn_rate: 0.07666, precision: 0.00957                                 
no_comp: 15, lrn_rate: 0.05658, precision: 0.01046                                 
no_comp: 10, lrn_rate: 0.07657, precision: 0.01016                                 
no_comp: 20, lrn_rate: 0.07264, precision: 0.01046                                 
no_comp: 15, lrn_rate: 0.09491, precision: 0.01061                                 
no_comp: 15, lrn_rate: 0.09454, precision: 0.01031                                 
no_comp: 20, lrn_rate: 0.07134, precision: 0.00882                                 
100%|██████████| 10/10 [00:03<00:00,  2.88trial/s, best loss: -0.010762332007288933]


In [507]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.06 2


In [508]:
learning_rate = 0.03
no_components = 3

In [509]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 68.03it/s]


In [510]:
# 모델 저장하기
import pickle
with open('model_color_onepiece.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [511]:
# 모델 불러오기
import pickle
with open('model_color_onepiece.pickle', 'rb') as f:
    model_onepiece = pickle.load(f)

In [512]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [513]:
rec_size_top = sample_recommendation(model_onepiece, cloth_meta.idx, real_user)
rec_size_top

[14435003027,
 21430112964,
 22207502998,
 19186162758,
 17093152675,
 21101903074,
 20163412966,
 19345122958,
 23792962867,
 18594522933]

In [514]:
real_user

[10780, 12086, 13922]

In [515]:
cloth_meta.insert(18, 'what', 9, True)
user_meta.insert(17, 'what', 9, True)

In [516]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,14094532674,279,9
1,14163822698,1239,9
2,18962342742,78,9
3,14829532807,1135,9
4,13617652829,1543,9
...,...,...,...
2042,20653784693,298,9
2043,13105704695,1719,9
2044,14406794698,837,9
2045,19412954699,1421,9


In [517]:
cloth_meta.to_json('color_onepiece_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('color_onepiece_user_meta.json', orient='records', force_ascii=False)

In [518]:
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,what
0,8297,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
1,8298,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
2,8299,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
3,8300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9
4,8301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5832,14129,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
5833,14130,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
5834,14131,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
5835,14132,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


# Skirt

In [519]:
transaction = pd.read_json('./transaction_skirt.json')
cloth_meta = pd.read_json('./final_skirt.json')
user_meta = pd.read_json('./user_skirt.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [520]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [521]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [522]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [523]:
user_meta = user_meta[color_user_column]

In [524]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0.00,0.0,0.5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
2,2,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,0.00,0.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.25,0.0,0.5,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8292,8292,0.00,0.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8293,8293,1.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8294,8294,0.00,0.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8295,8295,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [525]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('colorWhite')] = 0.3
new_user.loc[new_user_idx, ('colorGrey')] = 0.3
new_user.loc[new_user_idx, ('colorBlack')] = 0.4
new_user.loc[new_user_idx, ('colorRed')] = 0
new_user.loc[new_user_idx, ('colorPink')] = 0
new_user.loc[new_user_idx, ('colorOrange')] = 0
new_user.loc[new_user_idx, ('colorIvory')] = 0
new_user.loc[new_user_idx, ('colorYellow')] = 0
new_user.loc[new_user_idx, ('colorGreen')] = 0
new_user.loc[new_user_idx, ('colorBlue')] = 0
new_user.loc[new_user_idx, ('colorPurple')] = 0
new_user.loc[new_user_idx, ('colorBrown')] = 0
new_user.loc[new_user_idx, ('colorBeige')] = 0
new_user.loc[new_user_idx, ('colorJean')] = 0
new_user.loc[new_user_idx, ('colorPattern')] = 0
new_user.loc[new_user_idx, ('colorOthers')] = 0
new_user = new_user[['userId',  'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers']]
new_user

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,8299.0,0.3,0.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [526]:
colum = list(new_user.columns)
di = dict()
for i in range(len(colum)):
    di[colum[i]] = new_user[colum[i]][0]
col_list = []
for idx, key in enumerate(di):
    if idx > 0 and di[key] > 0.2:
        col_list.append(key)
col_list.insert(0, 'userId')
print(col_list)

['userId', 'colorWhite', 'colorGrey', 'colorBlack']


In [528]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [529]:
for i in range(1, len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)

In [530]:
sub_col = list(sub_user.columns)[1:]

In [531]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sum(sub_user[col_i] for col_i in sub_col)
sub_user = sub_user.sort_values('favorSum')

In [532]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[2669, 6471, 1863]


In [533]:
cloth_meta = cloth_meta[color_cloth_column]

In [534]:
cloth_meta

,newClothId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,2018790,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1844574,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1672093,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1687533,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2338446,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8871,21172562649,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8872,21172562648,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8873,23400702652,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8874,23400702653,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [535]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [536]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

2654

In [537]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [538]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [539]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [540]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 8297
clothId의 유니크 개수 : 2654
user테이블의 userId의 유니크 개수 : 8297
cloth테이블의 clothId의 유니크 개수 : 2654


In [541]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['colorWhite'][i], user_meta['colorGrey'][i], user_meta['colorBlack'][i],
                        user_meta['colorRed'][i], user_meta['colorPink'][i], user_meta['colorOrange'][i],
                        user_meta['colorIvory'][i], user_meta['colorYellow'][i], user_meta['colorGreen'][i],
                        user_meta['colorBlue'][i], user_meta['colorPurple'][i], user_meta['colorBrown'][i], user_meta['colorBeige'][i],
                        user_meta['colorJean'][i], user_meta['colorPattern'][i], user_meta['colorOthers'][i]]) for i in range(user_meta.shape[0])]

In [542]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['colorWhite'][i], cloth_meta['colorGrey'][i], cloth_meta['colorBlack'][i],
                        cloth_meta['colorRed'][i], cloth_meta['colorPink'][i], cloth_meta['colorOrange'][i],
                        cloth_meta['colorIvory'][i], cloth_meta['colorYellow'][i], cloth_meta['colorGreen'][i],
                        cloth_meta['colorBlue'][i], cloth_meta['colorPurple'][i], cloth_meta['colorBrown'][i], cloth_meta['colorBeige'][i],
                        cloth_meta['colorJean'][i], cloth_meta['colorPattern'][i], cloth_meta['colorOthers'][i]]) for i in range(cloth_meta.shape[0])]

In [543]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [544]:
interactions, weights = dataset.build_interactions(transaction_source)

In [545]:
user_features = dataset.build_user_features(user_features_source)

In [546]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [547]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [548]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [549]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [550]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 15, lrn_rate: 0.09202, precision: 0.01223    
no_comp: 15, lrn_rate: 0.07425, precision: 0.01316                                 
no_comp: 10, lrn_rate: 0.08593, precision: 0.01059                                 
no_comp: 15, lrn_rate: 0.08180, precision: 0.01161                                 
no_comp: 20, lrn_rate: 0.05757, precision: 0.01100                                 
no_comp: 20, lrn_rate: 0.06651, precision: 0.01007                                 
no_comp: 20, lrn_rate: 0.09316, precision: 0.01254                                 
no_comp: 10, lrn_rate: 0.08784, precision: 0.01161                                 
no_comp: 15, lrn_rate: 0.05468, precision: 0.01326                                 
no_comp: 10, lrn_rate: 0.08906, precision: 0.01285                                 
100%|██████████| 10/10 [00:05<00:00,  1.80trial/s, best loss: -0.013257965445518494]


In [551]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.055 1


In [552]:
learning_rate = 0.03
no_components = 3

In [553]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 47.61it/s]


In [554]:
# 모델 저장하기
import pickle
with open('model_color_skirt.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [555]:
# 모델 불러오기
import pickle
with open('model_color_skirt.pickle', 'rb') as f:
    model_skirt = pickle.load(f)

In [556]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [557]:
rec_size_top = sample_recommendation(model_skirt, cloth_meta.idx, real_user)
rec_size_top

[2017517424,
 150926840,
 1907111403,
 1618698367,
 1173636175,
 2152938406,
 1969751296,
 1618698368,
 2054486283,
 1775973474]

In [558]:
real_user

[2669, 6471, 1863]

In [559]:
cloth_meta.insert(18, 'what', 10, True)
user_meta.insert(17, 'what', 10, True)

In [560]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,161812522,1682,10
1,161812521,896,10
2,185310349,1662,10
3,1884867110,2304,10
4,1629303146,965,10
...,...,...,...
2649,21172562650,22,10
2650,21172562649,2,10
2651,21172562648,531,10
2652,23400702652,1441,10


In [561]:
cloth_meta.to_json('color_skirt_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('color_skirt_user_meta.json', orient='records', force_ascii=False)

In [562]:
user_meta

,userId,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,what
0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10
1,1,0.00,0.0,0.5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,10
2,2,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10
3,3,0.00,0.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,4,0.25,0.0,0.5,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8292,8292,0.00,0.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
8293,8293,1.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
8294,8294,0.00,0.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
8295,8295,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10
